# 逻辑回归

本笔记本实现了逻辑回归模型，包括决策边界可视化、成本函数和梯度下降的实现。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1. 生成二分类数据

In [ ]:
def generate_binary_classification_data():
    """
    生成二分类数据
    
    返回:
    X: 特征数据 (n_samples, 2)
    y: 标签 (n_samples, 1)，取值为0或1
    """
    n_samples = 100
    
    # 生成第一类数据
    X1 = np.random.randn(n_samples//2, 2) + np.array([2, 2])
    y1 = np.zeros((n_samples//2, 1))
    
    # 生成第二类数据
    X2 = np.random.randn(n_samples//2, 2) + np.array([-2, -2])
    y2 = np.ones((n_samples//2, 1))
    
    # 合并数据
    X = np.vstack((X1, X2))
    y = np.vstack((y1, y2))
    
    return X, y

# 生成数据
X, y = generate_binary_classification_data()
print(f"数据形状: X={X.shape}, y={y.shape}")

## 2. 可视化原始数据

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X[y[:, 0] == 0][:, 0], X[y[:, 0] == 0][:, 1], alpha=0.6, label='类别 0')
plt.scatter(X[y[:, 0] == 1][:, 0], X[y[:, 0] == 1][:, 1], alpha=0.6, label='类别 1')
plt.xlabel('特征 1')
plt.ylabel('特征 2')
plt.title('二分类数据')
plt.legend()
plt.grid(True)
plt.show()

## 3. 实现sigmoid函数

In [ ]:
def sigmoid(z):
    """
    sigmoid函数实现
    
    参数:
    z: 输入值
    
    返回:
    sigmoid(z): sigmoid函数值
    """
    return 1 / (1 + np.exp(-z))

# 可视化sigmoid函数
z = np.linspace(-10, 10, 100)
s = sigmoid(z)

plt.figure(figsize=(10, 6))
plt.plot(z, s)
plt.xlabel('z')
plt.ylabel('sigmoid(z)')
plt.title('Sigmoid函数')
plt.grid(True)
plt.show()

## 4. 实现逻辑回归的成本函数

In [ ]:
def compute_cost_logistic(X, y, theta):
    """
    计算逻辑回归的成本函数
    
    参数:
    X: 特征矩阵 (m, n+1)，包含偏置项
    y: 标签向量 (m, 1)
    theta: 参数向量 (n+1, 1)
    
    返回:
    cost: 成本函数值
    """
    m = len(y)
    h = sigmoid(X.dot(theta))
    
    # 计算成本函数
    cost = -(1/m) * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h))
    
    return cost

## 5. 实现逻辑回归的梯度下降

In [ ]:
def gradient_descent_logistic(X, y, theta, learning_rate, n_iterations):
    """
    使用梯度下降算法训练逻辑回归模型
    
    参数:
    X: 特征矩阵 (m, n+1)，包含偏置项
    y: 标签向量 (m, 1)
    theta: 初始参数向量 (n+1, 1)
    learning_rate: 学习率
    n_iterations: 迭代次数
    
    返回:
    theta: 学习后的参数向量
    cost_history: 每次迭代的成本函数值
    """
    m = len(y)
    cost_history = np.zeros(n_iterations)
    
    for i in range(n_iterations):
        # 计算预测值
        h = sigmoid(X.dot(theta))
        # 计算梯度
        gradient = (1/m) * X.T.dot(h - y)
        # 更新参数
        theta = theta - learning_rate * gradient
        # 记录成本函数值
        cost_history[i] = compute_cost_logistic(X, y, theta)
    
    return theta, cost_history

## 6. 训练逻辑回归模型

In [ ]:
# 添加偏置项
X_b = np.c_[np.ones((len(X), 1)), X]

# 初始化参数
theta_initial = np.zeros((3, 1))

# 设置超参数
learning_rate = 0.1
n_iterations = 1000

# 训练模型
theta, cost_history = gradient_descent_logistic(X_b, y, theta_initial, learning_rate, n_iterations)

# 打印学习结果
print(f"学习到的参数: theta0 = {theta[0][0]:.4f}, theta1 = {theta[1][0]:.4f}, theta2 = {theta[2][0]:.4f}")

## 7. 可视化训练过程

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(n_iterations), cost_history)
plt.xlabel('迭代次数')
plt.ylabel('成本函数值')
plt.title('逻辑回归的梯度下降训练过程')
plt.grid(True)
plt.show()

## 8. 可视化决策边界

In [ ]:
def plot_decision_boundary(X, y, theta):
    """
    绘制逻辑回归的决策边界
    
    参数:
    X: 特征数据 (m, 2)
    y: 标签 (m, 1)
    theta: 学习到的参数 (3, 1)
    """
    plt.figure(figsize=(10, 6))
    
    # 绘制数据点
    plt.scatter(X[y[:, 0] == 0][:, 0], X[y[:, 0] == 0][:, 1], alpha=0.6, label='类别 0')
    plt.scatter(X[y[:, 0] == 1][:, 0], X[y[:, 0] == 1][:, 1], alpha=0.6, label='类别 1')
    
    # 绘制决策边界 (sigmoid(z) = 0.5 时，z = 0)
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    
    xx1, xx2 = np.meshgrid(np.linspace(x1_min, x1_max, 100),
                           np.linspace(x2_min, x2_max, 100))
    
    # 生成网格点并预测
    grid = np.c_[np.ones((100*100, 1)), xx1.ravel(), xx2.ravel()]
    probs = sigmoid(grid.dot(theta)).reshape(xx1.shape)
    
    # 绘制决策边界 (概率为0.5的等高线)
    plt.contour(xx1, xx2, probs, levels=[0.5], linewidths=2, colors='red')
    
    plt.xlabel('特征 1')
    plt.ylabel('特征 2')
    plt.title('逻辑回归的决策边界')
    plt.legend()
    plt.grid(True)
    plt.show()

# 绘制决策边界
plot_decision_boundary(X, y, theta)

## 9. 模型预测与评估

In [ ]:
def predict(X, theta, threshold=0.5):
    """
    使用逻辑回归模型进行预测
    
    参数:
    X: 特征矩阵 (m, n+1)，包含偏置项
    theta: 学习到的参数
    threshold: 阈值，默认为0.5
    
    返回:
    predictions: 预测结果 (m, 1)
    """
    probabilities = sigmoid(X.dot(theta))
    return (probabilities >= threshold).astype(int)

# 预测训练数据
predictions = predict(X_b, theta)

# 计算准确率
accuracy = np.mean(predictions == y)
print(f"模型准确率: {accuracy:.4f}")